<a href="https://colab.research.google.com/github/gogamid/ml-notebooks/blob/main/continualai/05_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
description: Automatic Evaluation with Pre-implemented Metrics
---

# Evaluation

Welcome to the "_Evaluation_" tutorial of the "_From Zero to Hero_" series. In this part we will present the functionalities offered by the `evaluation` module.

In [1]:
!pip install avalanche-lib==0.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.8/452.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=f06b77a81c34eeb435c1fdf45b5091eb9b5c28aa16a1ccd3cb9dd67c1664ede6
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab

## 📈 The Evaluation Module



The `evaluation` module is quite straightforward: it offers all the basic functionalities to evaluate and keep track of a continual learning experiment.

This is mostly done through the **Metrics**: a set of classes which implement the main continual learning metrics computation like A_ccuracy_, F_orgetting_, M_emory Usage_, R_unning Times_, etc. At the moment, in _Avalanche_ we offer a number of pre-implemented metrics you can use for your own experiments. We made sure to include all the major accuracy-based metrics but also the ones related to computation and memory.

Each metric comes with a standalone class and a set of plugin classes aimed at emitting metric values on specific moments during training and evaluation.

#### Standalone metric

As an example, the standalone `Accuracy` class can be used to monitor the average accuracy over a stream of `<input,target>` pairs. The class provides an `update` method to update the current average accuracy, a `result` method to print the current average accuracy and a `reset` method to set the current average accuracy to zero. The call to `result`does not change the metric state.  

The `TaskAwareAccuracy` metric keeps separate accuracy counters for different task labels. As such, it requires the `task_labels` parameter, which specifies which task is associated with the current patterns. The metric returns a dictionary mapping task labels to accuracy values.

In [2]:
import torch
from avalanche.evaluation.metrics import Accuracy, TaskAwareAccuracy

# create an instance of the standalone Accuracy metric
# initial accuracy is 0
acc_metric = Accuracy()
print("Initial Accuracy: ", acc_metric.result()) #  output 0.0

# two consecutive metric updates
real_y = torch.tensor([1, 2]).long()
predicted_y = torch.tensor([1, 0]).float()
acc_metric.update(real_y, predicted_y)
acc = acc_metric.result()
print("Average Accuracy: ", acc) # output 0.5
predicted_y = torch.tensor([1,2]).float()
acc_metric.update(real_y, predicted_y)
acc = acc_metric.result()
print("Average Accuracy: ", acc) # output 0.75

predicted_y = torch.tensor([0,0]).float()
acc_metric.update(real_y, predicted_y)
acc = acc_metric.result()
print("Average Accuracy: ", acc) # output 0.5

# reset accuracy
acc_metric.reset()
print("After reset: ", acc_metric.result()) # output 0.0

Initial Accuracy:  0.0
Average Accuracy:  0.5
Average Accuracy:  0.75
Average Accuracy:  0.5
After reset:  0.0


In [3]:
# create an instance of the standalone TaskAwareAccuracy metric
# initial accuracy is 0 for each task
acc_metric = TaskAwareAccuracy()
print("Initial Accuracy: ", acc_metric.result()) #  output {}

# metric updates for 2 different tasks
task_label = 0
real_y = torch.tensor([1, 2]).long()
predicted_y = torch.tensor([1, 0]).float()
acc_metric.update(real_y, predicted_y, task_label)
acc = acc_metric.result()
print("Average Accuracy: ", acc) # output 0.5 for task 0

task_label = 1
predicted_y = torch.tensor([1,2]).float()
acc_metric.update(real_y, predicted_y, task_label)
acc = acc_metric.result()
print("Average Accuracy: ", acc) # output 0.75 for task 0 and 1.0 for task 1

task_label = 0
predicted_y = torch.tensor([1,2]).float()
acc_metric.update(real_y, predicted_y, task_label)
acc = acc_metric.result()
print("Average Accuracy: ", acc) # output 0.75 for task 0 and 1.0 for task 1

# reset accuracy
acc_metric.reset()
print("After reset: ", acc_metric.result()) # output {}

Initial Accuracy:  {}
Average Accuracy:  {0: 0.5}
Average Accuracy:  {0: 0.5, 1: 1.0}
Average Accuracy:  {0: 0.75, 1: 1.0}
After reset:  {}


#### Plugin metric

If you want to integrate the available metrics automatically in the training and evaluation flow, you can use plugin metrics, like `EpochAccuracy` which logs the accuracy after each training epoch, or `ExperienceAccuracy` which logs the accuracy after each evaluation experience. Each of these metrics emits a **curve** composed by its values at different points in time \(e.g. on different training epochs\).  In order to simplify the use of these metrics, we provided utility functions with which you can create different plugin metrics in one shot. The results of these functions can be passed as parameters directly to the `EvaluationPlugin`\(see below\).

{% hint style="info" %}
We recommend to use the helper functions when creating plugin metrics.
{% endhint %}

In [4]:
from avalanche.evaluation.metrics import accuracy_metrics, \
    loss_metrics, forgetting_metrics, bwt_metrics,\
    confusion_matrix_metrics, cpu_usage_metrics, \
    disk_usage_metrics, gpu_usage_metrics, MAC_metrics, \
    ram_usage_metrics, timing_metrics

# you may pass the result to the EvaluationPlugin
metrics = accuracy_metrics(epoch=True, experience=True)

## 📐Evaluation Plugin

The **Evaluation Plugin** is the object in charge of configuring and controlling the evaluation procedure. This object can be passed to a Strategy as a "special" plugin through the evaluator attribute.

The Evaluation Plugin accepts as inputs the plugin metrics you want to track. In addition, you can add one or more loggers to print the metrics in different ways \(on file, on standard output, on Tensorboard...\).

It is also recommended to pass to the Evaluation Plugin the benchmark instance used in the experiment. This allows the plugin to check for consistency during metrics computation. For example, the Evaluation Plugin checks that the `strategy.eval` calls are performed on the same stream or sub-stream. Otherwise, same metric could refer to different portions of the stream.  
These checks can be configured to raise errors (stopping computation) or only warnings.

In [5]:
# MY SIMPLE CUMULATIVE EXAMPLE with evaluation plugin showing just accuracy
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive, Cumulative

benchmark = SplitMNIST(n_experiences=5)

# MODEL CREATION
model = SimpleMLP(num_classes=benchmark.n_classes)

# DEFINE THE EVALUATION PLUGIN
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=False, experience=True, stream=True),
    loggers=[InteractiveLogger()],
    strict_checks=False
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
# you can use naive too
cl_strategy = Cumulative(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

100%|██████████| 9912422/9912422 [00:00<00:00, 73085887.84it/s]


Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 60964113.65it/s]

Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 25969716.78it/s]


Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11483139.70it/s]


Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw

Starting experiment...
-- >> Start of training phase << --
100%|██████████| 25/25 [00:03<00:00,  6.90it/s]
Epoch 0 ended.
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 21/21 [00:01<00:00, 18.88it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9116
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 20.50it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 21.08it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	To

In [6]:
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive

benchmark = SplitMNIST(n_experiences=5)

# MODEL CREATION
model = SimpleMLP(num_classes=benchmark.n_classes)

# DEFINE THE EVALUATION PLUGIN
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[InteractiveLogger()],
    strict_checks=False
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Starting experiment...
-- >> Start of training phase << --
100%|██████████| 25/25 [00:02<00:00,  9.95it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55555.4482
	DiskUsage_MB/train_phase/train_stream/Task000 = 55555.4482
	Loss_Epoch/train_phase/train_stream/Task000 = 1.1397
	Loss_MB/train_phase/train_stream/Task000 = 0.3706
	Time_Epoch/train_phase/train_stream/Task000 = 2.5126
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7156
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8989
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 21/21 [00:00<00:00, 36.97it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 95.0770
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 55555.4482
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.3139
	Top1_

## Implement your own metric

To implement a **standalone metric**, you have to subclass `Metric` class.

In [7]:
from avalanche.evaluation import Metric


# a standalone metric implementation
class MyStandaloneMetric(Metric[float]):
    """
    This metric will return a `float` value
    """
    def __init__(self):
        """
        Initialize your metric here
        """
        super().__init__()
        pass

    def update(self):
        """
        Update metric value here
        """
        pass

    def result(self, **kwargs) -> float:
        """
        Emit the metric result here
        """
        return 0

    def reset(self, **kwargs):
        """
        Reset your metric here
        """
        pass

 To implement a **plugin metric** you have to subclass `PluginMetric` class

In [8]:
from avalanche.evaluation import PluginMetric
from avalanche.evaluation.metrics import Accuracy
from avalanche.evaluation.metric_results import MetricValue
from avalanche.evaluation.metric_utils import get_metric_name


class MyPluginMetric(PluginMetric[float]):
    """
    This metric will return a `float` value after
    each training epoch
    """

    def __init__(self):
        """
        Initialize the metric
        """
        super().__init__()

        self._accuracy_metric = Accuracy()

    def reset(self, **kwargs) -> None:
        """
        Reset the metric
        """
        self._accuracy_metric.reset()

    def result(self, **kwargs) -> float:
        """
        Emit the result
        """
        return self._accuracy_metric.result()

    def after_training_iteration(self, strategy: 'PluggableStrategy') -> None:
        """
        Update the accuracy metric with the current
        predictions and targets
        """
        # task labels defined for each experience
        task_labels = strategy.experience.task_labels
        if len(task_labels) > 1:
            # task labels defined for each pattern
            task_labels = strategy.mb_task_id
        else:
            task_labels = task_labels[0]

        self._accuracy_metric.update(strategy.mb_output, strategy.mb_y,
                                     task_labels)

    def before_training_epoch(self, strategy: 'PluggableStrategy') -> None:
        """
        Reset the accuracy before the epoch begins
        """
        self.reset()

    def after_training_epoch(self, strategy: 'PluggableStrategy'):
        """
        Emit the result
        """
        return self._package_result(strategy)


    def _package_result(self, strategy):
        """Taken from `GenericPluginMetric`, check that class out!"""
        metric_value = self.accuracy_metric.result()
        add_exp = False
        plot_x_position = strategy.clock.train_iterations

        if isinstance(metric_value, dict):
            metrics = []
            for k, v in metric_value.items():
                metric_name = get_metric_name(
                    self, strategy, add_experience=add_exp, add_task=k)
                metrics.append(MetricValue(self, metric_name, v,
                                           plot_x_position))
            return metrics
        else:
            metric_name = get_metric_name(self, strategy,
                                          add_experience=add_exp,
                                          add_task=True)
            return [MetricValue(self, metric_name, metric_value,
                                plot_x_position)]

    def __str__(self):
        """
        Here you can specify the name of your metric
        """
        return "Top1_Acc_Epoch"

## Accessing metric values

If you want to access all the metrics computed during training and evaluation, you have to make sure that `collect_all=True` is set when creating the `EvaluationPlugin` (default option is `True`). This option maintains an updated version of all metric results in the plugin, which can be retrieved by calling `evaluation_plugin.get_all_metrics()`. You can call this methods whenever you need the metrics.

The result is a dictionary with full metric names as keys and a tuple of two lists as values. The first list stores all the `x` values recorded for that metric. Each `x` value represents the time step at which the corresponding metric value has been computed. The second list stores metric values associated to the corresponding `x` value.

In [9]:
eval_plugin2 = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    timing_metrics(epoch=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    collect_all=True, # this is default value anyway
    loggers=[InteractiveLogger()]
)

# since no training and evaluation has been performed, this will return an empty dict.
metric_dict = eval_plugin2.get_all_metrics()
print(metric_dict)

defaultdict(<function _init_metrics_list_lambda at 0x7853d9053880>, {})


In [10]:
d = eval_plugin.get_all_metrics()
d['Top1_Acc_Epoch/train_phase/train_stream/Task000']

([25, 49, 73, 97, 123],
 [0.7156092315327984,
  0.47693092021412187,
  0.34733869587540644,
  0.23628190993130355,
  0.42250296091591])

Alternatively, the `train` and `eval` method of every `strategy` returns a dictionary storing, for each metric, the last value recorded for that metric. You can use these dictionaries to incrementally accumulate metrics.

In [11]:
print(res)

{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.9939393939393939, 'Loss_MB/train_phase/train_stream/Task000': 0.5233470797538757, 'DiskUsage_MB/train_phase/train_stream/Task000': 55555.4482421875, 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.42250296091591, 'Loss_Epoch/train_phase/train_stream/Task000': 1.9481726124547378, 'Time_Epoch/train_phase/train_stream/Task000': 2.7174827160000063, 'DiskUsage_Epoch/train_phase/train_stream/Task000': 55555.4482421875, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.0, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 2.7210617399355805, 'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000': 94.45590309456857, 'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000': 55555.4482421875, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.06884057971014493, 'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 2.4402871679815448, 'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001': 92.90756259760283, 'DiskUsage_Exp/eval_phase/test_str

In [12]:
print(results[-1])

{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.9939393939393939, 'Loss_MB/train_phase/train_stream/Task000': 0.5233470797538757, 'DiskUsage_MB/train_phase/train_stream/Task000': 55555.4482421875, 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.42250296091591, 'Loss_Epoch/train_phase/train_stream/Task000': 1.9481726124547378, 'Time_Epoch/train_phase/train_stream/Task000': 2.7174827160000063, 'DiskUsage_Epoch/train_phase/train_stream/Task000': 55555.4482421875, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.0, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 2.843151956300427, 'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000': 96.36418486771937, 'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000': 55555.4482421875, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.0005175983436853002, 'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 2.918635980929894, 'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001': 95.93367377560546, 'DiskUsage_Exp/eval_phase/test_str

This completes the "_Evaluation_" tutorial for the "_From Zero to Hero_" series. We hope you enjoyed it!

## 🤝 Run it on Google Colab

You can run _this chapter_ and play with it on Google Colaboratory: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContinualAI/avalanche/blob/master/notebooks/from-zero-to-hero-tutorial/05_evaluation.ipynb)